# A star demo with py game

In [1]:
import pygame
import math
from queue import PriorityQueue

WIDTH = 800
WIN = pygame.display.set_mode((WIDTH, WIDTH))
pygame.display.set_caption("A* Path Finding Algorithm")

RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165 ,0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)

class Spot:
	def __init__(self, row, col, width, total_rows):
		self.row = row
		self.col = col
		self.x = row * width
		self.y = col * width
		self.color = WHITE
		self.neighbors = []
		self.width = width
		self.total_rows = total_rows

	def get_pos(self):
		return self.row, self.col

	def is_closed(self):
		return self.color == RED

	def is_open(self):
		return self.color == GREEN

	def is_barrier(self):
		return self.color == BLACK

	def is_start(self):
		return self.color == ORANGE

	def is_end(self):
		return self.color == TURQUOISE

	def reset(self):
		self.color = WHITE

	def make_start(self):
		self.color = ORANGE

	def make_closed(self):
		self.color = RED

	def make_open(self):
		self.color = GREEN

	def make_barrier(self):
		self.color = BLACK

	def make_end(self):
		self.color = TURQUOISE

	def make_path(self):
		self.color = PURPLE

	def draw(self, win):
		pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

	def update_neighbors(self, grid):
		self.neighbors = []
		if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN
			self.neighbors.append(grid[self.row + 1][self.col])

		if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP
			self.neighbors.append(grid[self.row - 1][self.col])

		if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT
			self.neighbors.append(grid[self.row][self.col + 1])

		if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT
			self.neighbors.append(grid[self.row][self.col - 1])

	def __lt__(self, other):
		return False


def h(p1, p2):
	x1, y1 = p1
	x2, y2 = p2
	return abs(x1 - x2) + abs(y1 - y2)


def reconstruct_path(came_from, current, draw):
	while current in came_from:
		current = came_from[current]
		current.make_path()
		draw()


def algorithm(draw, grid, start, end):
	count = 0
	open_set = PriorityQueue()
	open_set.put((0, count, start))
	came_from = {}
	g_score = {spot: float("inf") for row in grid for spot in row}
	g_score[start] = 0
	f_score = {spot: float("inf") for row in grid for spot in row}
	f_score[start] = h(start.get_pos(), end.get_pos())

	open_set_hash = {start}

	while not open_set.empty():
		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				pygame.quit()

		current = open_set.get()[2]
		open_set_hash.remove(current)

		if current == end:
			reconstruct_path(came_from, end, draw)
			end.make_end()
			return True

		for neighbor in current.neighbors:
			temp_g_score = g_score[current] + 1

			if temp_g_score < g_score[neighbor]:
				came_from[neighbor] = current
				g_score[neighbor] = temp_g_score
				f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
				if neighbor not in open_set_hash:
					count += 1
					open_set.put((f_score[neighbor], count, neighbor))
					open_set_hash.add(neighbor)
					neighbor.make_open()

		draw()

		if current != start:
			current.make_closed()

	return False


def make_grid(rows, width):
	grid = []
	gap = width // rows
	for i in range(rows):
		grid.append([])
		for j in range(rows):
			spot = Spot(i, j, gap, rows)
			grid[i].append(spot)

	return grid


def draw_grid(win, rows, width):
	gap = width // rows
	for i in range(rows):
		pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
		for j in range(rows):
			pygame.draw.line(win, GREY, (j * gap, 0), (j * gap, width))


def draw(win, grid, rows, width):
	win.fill(WHITE)

	for row in grid:
		for spot in row:
			spot.draw(win)

	draw_grid(win, rows, width)
	pygame.display.update()


def get_clicked_pos(pos, rows, width):
	gap = width // rows
	y, x = pos

	row = y // gap
	col = x // gap

	return row, col


def main(win, width):
	ROWS = 50
	grid = make_grid(ROWS, width)

	start = None
	end = None

	run = True
	while run:
		draw(win, grid, ROWS, width)
		for event in pygame.event.get():
			if event.type == pygame.QUIT:
				run = False

			if pygame.mouse.get_pressed()[0]: # LEFT
				pos = pygame.mouse.get_pos()
				row, col = get_clicked_pos(pos, ROWS, width)
				spot = grid[row][col]
				if not start and spot != end:
					start = spot
					start.make_start()

				elif not end and spot != start:
					end = spot
					end.make_end()

				elif spot != end and spot != start:
					spot.make_barrier()

			elif pygame.mouse.get_pressed()[2]: # RIGHT
				pos = pygame.mouse.get_pos()
				row, col = get_clicked_pos(pos, ROWS, width)
				spot = grid[row][col]
				spot.reset()
				if spot == start:
					start = None
				elif spot == end:
					end = None

			if event.type == pygame.KEYDOWN:
				if event.key == pygame.K_SPACE and start and end:
					for row in grid:
						for spot in row:
							spot.update_neighbors(grid)

					algorithm(lambda: draw(win, grid, ROWS, width), grid, start, end)

				if event.key == pygame.K_c:
					start = None
					end = None
					grid = make_grid(ROWS, width)

	pygame.quit()

main(WIN, WIDTH)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
from tkinter import *
from PIL import Image, ImageTk
import random
import itertools 
App = Tk()
App.geometry('650x600')
App.title('Vacumm simulator')

global grid, current_position, goal_position 



def dfs_search(grid, current_position, goal_position):
    rows, cols = len(grid), len(grid[0])
    visited = [[False for _ in range(cols)] for _ in range(rows)] 
    visited_nodes = [] # to keep track of visited nodes
    def dfs_helper(x, y):
        if x < 0 or x >= rows or y < 0 or y >= cols or visited[x][y] or grid[x][y] == 1:
            return None

        visited[x][y] = True
        visited_nodes.append((x, y)) # add node to visited nodes

        if (x, y) == goal_position:
            return [(x, y)]

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            result = dfs_helper(new_x, new_y)
            if result:
                return [(x, y)] + result

        return None

    path = dfs_helper(current_position[0], current_position[1])
    print("Path found:", path)
    return visited_nodes, path, step # return visited nodes if no path found

def handle_start(canvas,rectangle_ids):
    global current_position
    global goal_position

    visited_nodes, path, step = dfs_search(grid, current_position, goal_position)
    for i, step in enumerate(visited_nodes):
        if i > 0:
            current_position = step
            move =  current_position[0]* len(grid) + current_position[1] 
            canvas.itemconfig(rectangle_ids[move], fill="yellow")
            App.update()  
            App.after(100)
    # step_label = Label(main_frame, text=f"Steps taken: {step}")
    # step_label.pack()    
    if not path:
        not_found = Label(main_frame, text="Solution not found")
        not_found.pack               

def draw_grid(page, number):
    global grid, current_position, goal_position
    pairs = []
    for i in range(number):
        pair = [random.randint(0, number-1), random.randint(0, number-1)]
        while pair in [[0, 0], [number-1, number-1]]:
            pair = [random.randint(0, number-1), random.randint(0, number-1)]
        pairs.append(pair)
    canvas = Canvas(page, width=400, height=400, bg="#76ABAE")
    step = 400  // number
    rectangle_ids = []
    grid = [[0 for _ in range(number)] for _ in range(number)]
    goal_position = (len(grid)-1, len(grid)-1)
    for x in range(0,400 // step , 1):
        for y in range(0, 400 // step, 1):
            obstacle = False
            for pair in pairs:  
                if(x == pair[0] and y == pair[1]):
                    rectangle_id = canvas.create_rectangle(x * step,y * step,x*step + step,y*step + step, fill="#9B3922")
                    grid[x][y] = 1
                    obstacle = True
                    break
            if(x == 0 and y == 0):
                rectangle_id = canvas.create_rectangle(x * step,y * step,x*step + step,y*step + step, fill="yellow")
                current_position = (x,y)
            elif(x == 400 // step -1  and y == 400 // step - 1):
                rectangle_id = canvas.create_rectangle(x * step,y * step,x*step + step,y*step + step, fill="blue")
            elif(obstacle == False):     
                rectangle_id =  canvas.create_rectangle(x * step,y * step,x*step + step,y*step + step)
            rectangle_ids.append(rectangle_id)         
                    
    canvas.pack()
    start_btn = Button(page,text="START", fg="#76ABAE", font=("Bold",15), border=0 ,command=lambda:handle_start(canvas, rectangle_ids))
    start_btn.pack()


def frame_4():
    frame_4 = Frame(main_frame)
    draw_grid(frame_4,4)
    frame_4.pack(pady=10)

def frame_8():
    frame_8 = Frame(main_frame)
    draw_grid(frame_8,8)
    frame_8.pack(pady=10)


def frame_16():
    frame_16 = Frame(main_frame)
    draw_grid(frame_16,16)
    frame_16.pack(pady=10)



def hide_indicator():
    button_4_indicator.config(bg ="#31363F")
    button_8_indicator.config(bg ="#31363F")
    button_16_indicator.config(bg ="#31363F")

def delete_frame():
    for frame in main_frame.winfo_children():
        frame.destroy()


def indicate(label, page):
    hide_indicator()
    label.config(bg = "#76ABAE")
    delete_frame()
    page()

side_bar = Frame(App, bg="#31363F")
side_bar.pack(side=LEFT)
side_bar.pack_propagate(False)
side_bar.configure(width=150,height=600)

select_label = Label(side_bar, text="select size", font=("Bold",15), fg="#76ABAE", bg="#31363F")
select_label.place(x=5, y= 20)

button_4 = Button(side_bar, text="4 x 4", font=("Bold",10), fg="#76ABAE", bd=0, bg="#31363F", command=lambda:indicate(button_4_indicator, frame_4))
button_4.place(x=25, y= 50)
button_4_indicator = Label(side_bar, text="",bg="#31363F")
button_4_indicator.place(x = 5, y = 45, width=5, height=30)




button_8 = Button(side_bar, text="8 x 8", font=("Bold",10), fg="#76ABAE", bd=0, bg="#31363F", command=lambda:indicate(button_8_indicator, frame_8))
button_8.place(x=25, y= 100)
button_8_indicator = Label(side_bar, text="",bg="#31363F")
button_8_indicator.place(x = 5, y = 95, width=5, height=30)





button_16 = Button(side_bar, text="16 x 16", font=("Bold",10), fg="#76ABAE", bd=0, bg="#31363F", command=lambda:indicate(button_16_indicator, frame_16))
button_16.place(x=25, y= 150)
button_16_indicator = Label(side_bar, text="",bg="#31363F")
button_16_indicator.place(x = 5, y = 145, width=5, height=30)



main_frame = Frame(App, highlightbackground='black',highlightthickness=2, bg="#222831")
main_frame.pack(side=LEFT)
main_frame.pack_propagate(False)
main_frame.configure(width=500,height=600)



App.mainloop()